## Data Loading

In [1]:
import h5py
import numpy as np
import pickle

data = h5py.File('/home/sunji/ANN/glove_200_angular/glove-200-angular.hdf5', 'r')
data_train = np.array(data['train'])
data_test = np.array(data['test'])
with open('/home/sunji/ANN/glove_200_angular/clusters_glove_200_angular.pkl', 'rb') as f:
    clusters = pickle.load(f)
with open('/home/sunji/ANN/glove_200_angular/ground_truth_glove_200_angular_0_4_0_5.pkl', 'rb') as f:
    ground_truth_total = pickle.load(f)

In [2]:
for idx, c in enumerate(clusters):
    print (idx, len(c))

0 21002
1 7585
2 20072
3 20501
4 9835
5 7053
6 4612
7 8862
8 6927
9 15750
10 12257
11 4633
12 23344
13 7865
14 17241
15 8881
16 7432
17 21209
18 4285
19 4884
20 5573
21 12063
22 8629
23 9347
24 13000
25 18926
26 11938
27 2008
28 9458
29 20117
30 5167
31 24637
32 16506
33 4974
34 9768
35 24978
36 11416
37 10900
38 9872
39 4392
40 8858
41 17615
42 5080
43 12512
44 6673
45 6482
46 10698
47 6538
48 15843
49 10325
50 6219
51 26745
52 28277
53 23462
54 10349
55 9829
56 1290
57 7895
58 5464
59 5025
60 15746
61 10512
62 15796
63 2160
64 1720
65 13290
66 19456
67 7105
68 8093
69 20274
70 7049
71 13444
72 4270
73 14803
74 13106
75 7345
76 18853
77 7240
78 13331
79 10130
80 6414
81 17541
82 5768
83 11370
84 9801
85 1750
86 9711
87 17169
88 12820
89 15282
90 25213
91 11949
92 22146
93 26562
94 5596
95 13140
96 4932
97 12521
98 24137
99 12891


In [3]:
ground_truth_total_level = [[[] for _ in range(10000)] for _ in range(100)]
for clus in range(100):
    for t in ground_truth_total[clus]:
        ground_truth_total_level[t[0]][t[1]].append(t)

In [25]:
ground_truth_total_level[0]

[[(0, 0, 0.4, 0.402, 10),
  (0, 0, 0.402, 0.404, 11),
  (0, 0, 0.404, 0.406, 7),
  (0, 0, 0.406, 0.40800000000000003, 7),
  (0, 0, 0.40800000000000003, 0.41000000000000003, 14),
  (0, 0, 0.41000000000000003, 0.41200000000000003, 22),
  (0, 0, 0.41200000000000003, 0.41400000000000003, 32),
  (0, 0, 0.41400000000000003, 0.41600000000000004, 30),
  (0, 0, 0.41600000000000004, 0.41800000000000004, 38),
  (0, 0, 0.41800000000000004, 0.42000000000000004, 36),
  (0, 0, 0.42000000000000004, 0.42200000000000004, 44),
  (0, 0, 0.42200000000000004, 0.42400000000000004, 57),
  (0, 0, 0.42400000000000004, 0.42600000000000005, 58),
  (0, 0, 0.42600000000000005, 0.42800000000000005, 59),
  (0, 0, 0.42800000000000005, 0.43000000000000005, 90),
  (0, 0, 0.43000000000000005, 0.43200000000000005, 104),
  (0, 0, 0.43200000000000005, 0.43400000000000005, 126),
  (0, 0, 0.43400000000000005, 0.43600000000000005, 164),
  (0, 0, 0.43600000000000005, 0.43800000000000006, 151),
  (0, 0, 0.43800000000000006, 0.44

In [4]:
centroids = []
for cluster in clusters:
    centroids.append(np.mean(cluster, 0))

In [13]:
centroids[0]

array([39.254555 , 20.19043  , 17.16189  , 18.21472  , 16.192616 ,
        9.116712 ,  7.9420695, 16.78941  , 60.255283 , 49.224678 ,
       42.54615  , 36.685085 , 19.763906 ,  9.267671 ,  8.435876 ,
       19.159824 , 34.706825 , 34.554165 , 39.117805 , 51.008503 ,
       37.743137 , 13.057201 ,  9.600194 , 14.228443 , 31.790989 ,
       19.323416 , 15.472674 , 19.479353 , 22.413164 , 11.211683 ,
        8.20719  , 14.3472185, 69.901505 , 34.811028 , 23.6778   ,
       21.617804 , 21.691402 , 12.412679 , 11.835196 , 32.474495 ,
       89.21618  , 62.603107 , 44.806778 , 31.772528 , 17.742167 ,
        7.2682776,  9.2458105, 37.8573   , 30.031576 , 31.442312 ,
       42.384502 , 66.37321  , 70.46126  , 28.075054 , 11.682657 ,
       13.066311 , 42.77872  , 22.482025 , 19.898592 , 32.119747 ,
       45.893734 , 24.485426 , 14.938911 , 23.083555 , 67.05939  ,
       23.111124 , 12.086957 , 14.872237 , 22.153389 , 19.323416 ,
       24.871752 , 43.94705  , 83.85123  , 25.280544 ,  8.9540

## Prepare Inputs

In [5]:
import numpy as np
from numpy import dot
from numpy.linalg import norm
from scipy import spatial
import random
from multiprocessing import Pool

def angular_dist(x1, x2=None, eps=1e-8):
    cosine_sim = 1 - spatial.distance.cosine(x1, x2)
    distance = np.arccos(cosine_sim) / 3.14159267
    return distance 

def euclidean_dist_normalized(x1, x2=None, eps=1e-8):
    if x2 is None:
        return 1.0
    return np.sqrt(((x1 - x2) ** 2).mean())

def prepare_for_cluster(cluster_id):
    slot = 0.002
    batch_size = 128
    min_card = 1e10
    max_card = 0
    train_queries = []
    train_distances = []
    train_thresholds = []
    train_targets = []
    for query_id in range(8000):
        cardinality = 0
        for threshold_id, threshold in enumerate(np.arange(0.4, 0.5, slot)):
            cardinality += ground_truth_total_level[cluster_id][query_id][threshold_id][-1]
            if cardinality > 0 and random.random() < 0.33:
                train_queries.append(data_test[query_id])
                train_distances.append([angular_dist(data_test[query_id], centroids[cluster_id])])
                train_thresholds.append([threshold+slot])
                train_targets.append([cardinality])

    test_queries = []
    test_distances = []
    test_thresholds = []
    test_targets = []
    for query_id in range(8000,10000):
        cardinality = 0
        for threshold_id, threshold in enumerate(np.arange(0.4, 0.5, slot)):
            cardinality += ground_truth_total_level[cluster_id][query_id][threshold_id][-1]
            if cardinality > 0 and random.random() < 0.33:
                test_queries.append(data_test[query_id])
                test_distances.append([angular_dist(data_test[query_id], centroids[cluster_id])])
                test_thresholds.append([threshold+slot])
                test_targets.append([cardinality])
    
    train_loader = torch.utils.data.DataLoader(
        torch.utils.data.TensorDataset(torch.FloatTensor(train_queries), torch.FloatTensor(train_distances), torch.FloatTensor(train_thresholds), torch.FloatTensor(train_targets)), batch_size=batch_size, shuffle=True)
    test_loader = torch.utils.data.DataLoader(
        torch.utils.data.TensorDataset(torch.FloatTensor(test_queries), torch.FloatTensor(test_distances), torch.FloatTensor(test_thresholds), torch.FloatTensor(test_targets)), batch_size=batch_size, shuffle=True)
    
    return train_loader, test_loader, min_card, max_card
        

In [6]:
import torch
import torch.utils.data

train_loaders = []
test_loaders = []
min_cards = []
max_cards = []
for cluster_id in range(100):
    print (cluster_id)
    train, test, min_card, max_card = prepare_for_cluster(cluster_id)
    train_loaders.append(train)
    test_loaders.append(test)
    min_cards.append(min_card)
    max_cards.append(max_card)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [ ]:
def normalize(labels, mini, maxi):
    return (torch.log(labels) - mini) / (maxi - mini)

def unnormalize(labels, mini, maxi):
    return torch.exp(labels * (maxi - mini) + mini)


## BaseLine Local Model

In [7]:
from __future__ import print_function
import argparse
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

queries_dimension = 200
hidden_num = 128

class Test_Model(nn.Module):
    
    def __init__(self):
        super(Test_Model, self).__init__()
        self.threshold1 = nn.Linear(1, hidden_num)
        self.threshold2 = nn.Linear(hidden_num, 1)

        self.cnn_layer1 = nn.Sequential(
            nn.Conv1d(1, 8, kernel_size=3, stride=1, padding=2), 
            nn.BatchNorm1d(8),
            nn.LeakyReLU(),
            nn.AvgPool1d(kernel_size=3, stride=3))
        
        self.cnn_layer2 = nn.Sequential(
            nn.Conv1d(8, 16, kernel_size=3, stride=1, padding=2), 
            nn.BatchNorm1d(16),
            nn.LeakyReLU(),
            nn.AvgPool1d(kernel_size=3, stride=3))
        
        self.out1 = nn.Linear(368, hidden_num)
        self.out2 = nn.Linear(hidden_num, 1)
        
    def forward(self, query, threshold):
        
        query = query.unsqueeze(2).permute(0,2,1)
#         print (query.shape)
        threshold = F.relu(self.threshold1(threshold))
        threshold = self.threshold2(threshold)
#         print (threshold.shape)
        query = self.cnn_layer1(query)
        query = self.cnn_layer2(query)
        query = query.view(query.shape[0], -1)
#         print (query.shape)
        query = self.out1(query)
        
        output = self.out2(F.relu(query+threshold))
        
        return output

def begin_test_model():
#     models = []
#     errors = []
#     for idx in range(100):
    idx = 0
    print ('idx: {}'.format(idx))
    train = train_loaders[idx]
    test = test_loaders[idx]
    mini = min_cards[idx]
    maxi = max_cards[idx]
    episode = 5
    queries_dimension = 128
    model = Test_Model()
    opt = optim.Adam(model.parameters(), lr=0.01)
    error = test_model(model, opt, train, test, episode)
    models.append(model)
    errors.append(error)

def test_model(model, opt, train, test, episode):
    print ('size: {}'.format(len(train)))
    test_errors = []
    for e in range(episode):
        model.train()
        for batch_idx, (queries, _, thresholds, targets) in enumerate(train):
    #         print (torch.cat((queries, thresholds), 1)[0])
            queries = Variable(queries)
            thresholds = Variable(thresholds)
            targets = Variable(targets)
    #         print (targets)
            opt.zero_grad()
            
            estimates = model(queries, thresholds)
            
            loss = l1_loss(estimates, targets)
            loss.backward()
            opt.step()
#             if batch_idx % 100 == 0:
#                 print('Training: Iteration {0}, Batch {1}, Loss {2}'.format(e, batch_idx, loss.item()))
#                 print(model.cnn_layer1[0].weight)
            for p in model.parameters():
                p.data.clamp_(-2, 2)
        model.eval()
        test_loss = 0.0
        mse_error = 0.0
        q_mean = 0.0
        q_max = 0.0
        for batch_idx, (queries, _, thresholds, targets) in enumerate(test):
            queries = Variable(queries)
            thresholds = Variable(thresholds)
            targets = Variable(targets)
            
            estimates = model(queries, thresholds)
            
            loss = l1_loss(estimates, targets)
            mse, qer_mean, qer_max = print_loss(estimates, targets)
            test_loss += loss.item()
            mse_error += mse.item()
            q_mean += qer_mean
            if qer_max > q_max:
                q_max = qer_max
        test_loss /= len(test)
        mse_error /= len(test)
        q_mean /= len(test)
        test_errors.append(q_mean)
        print ('Testing: Iteration {0}, Loss {1}, MSE_error {2}, Q_error_mean {3}, Q_error_max {4}'.format(e, test_loss, mse_error, q_mean, q_max))
    return np.mean(test_errors[-3:])

In [ ]:
begin_test_model()

## Stacked Model Definition

In [8]:
from __future__ import print_function
import argparse
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

# queries_dimension = 200
# hidden_num = 128

class Threshold_Model(nn.Module):
    
    def __init__(self):
        super(Threshold_Model, self).__init__()
        self.fc1 = nn.Linear(1, hidden_num)
        self.fc2 = nn.Linear(hidden_num, 1)
    
    def forward(self, threshold):
        t1 = F.relu(self.fc1(threshold))
        t2 = self.fc2(t1)
        return t2

class CNN_Model(nn.Module):
    
    def __init__(self, in_channel, out_channel, kernel_size, stride, padding, pool_type, pool_size):
        super(CNN_Model, self).__init__()
        if pool_type == 0:
            pool_layer = nn.MaxPool1d(kernel_size=pool_size, stride=pool_size)
        elif pool_type == 1:
            pool_layer = nn.AvgPool1d(kernel_size=pool_size, stride=pool_size)
        else:
            print ('CNN_Model Init Error, invalid pool_type {}'.format(pool_type))
            return
        self.layer = nn.Sequential(
            nn.Conv1d(in_channel, out_channel, kernel_size=kernel_size, stride=stride, padding=padding), 
            nn.BatchNorm1d(out_channel),
            nn.ReLU(),
            pool_layer)
        
    def forward(self, inputs):
        hid = self.layer(inputs)
#         print (hid.shape)
#         hid = F.relu(self.n3(hid))
#         hid = F.relu(self.n4(hid))
#         hid = self.norm2(hid)
#         print (hid.shape)
#         out2 = self.fc(hid.view(out1.shape[0], -1))
        return hid

class Output_Model(nn.Module):
    
    def __init__(self, inputs_dim):
        super(Output_Model, self).__init__()
        self.fc1 = nn.Linear(inputs_dim, hidden_num)
        self.fc2 = nn.Linear(hidden_num, 1)
        
    def forward(self, queries, threshold):
        out1 = F.relu(self.fc1(queries))
        out2 = out1 + threshold
#         print ('out2: {0}, threshold: {1}'.format(out2.shape, threshold.shape))
        out3 = self.fc2(out2)
        return out3

class Model(nn.Module):
    
    def __init__(self):
        super(Model, self).__init__()
        self.nn1 = nn.Linear(queries_dimension+1, hidden_num)
        self.n1 = nn.Linear(hidden_num, hidden_num)
        self.n2 = nn.Linear(hidden_num, hidden_num)
#         self.n3 = nn.Linear(hidden_num, hidden_num)
#         self.n4 = nn.Linear(hidden_num, hidden_num)
        self.nn2 = nn.Linear(hidden_num, 1)
        
    def forward(self, queries, threshold):
        out1 = F.relu(self.nn1(torch.cat([queries, threshold],1)))
        hid = out1
        hid = F.relu(self.n1(hid))
        hid = F.relu(self.n2(hid))
#         hid = F.relu(self.n3(hid))
#         hid = F.relu(self.n4(hid))
#         hid = self.norm2(hid)
        out2 = self.nn2(hid)
        return out2

def loss_fn(estimates, targets, mini, maxi):
    est = unnormalize(estimates, mini, maxi)
    print (torch.cat((est, targets), 1))
    return F.mse_loss(est, targets)

def l1_loss(estimates, targets, eps=1e-5):
    estimates = torch.exp(estimates)
    qerror = 0.0
    for i in range(estimates.shape[0]):
        if estimates[i] > targets[i] + 0.1:
            qerror += ((estimates[i] / (targets[i] + 0.1)))
        else:
            qerror += (((targets[i] + 0.1) / estimates[i]))
    return qerror / estimates.shape[0]

def mse_loss(estimates, targets, eps=1e-5):
#     print (torch.cat((estimates, targets), 1))
    return F.mse_loss(estimates, torch.log(targets))

def qerror_loss(preds, targets, mini, maxi):
    qerror = []
    preds = unnormal1ize_label(preds, mini, maxi)
#     print (torch.cat((preds, targets), 1))
    for i in range(len(targets)):
        if (preds[i] > targets[i]).cpu().data.numpy()[0]:
            qerror.append(preds[i]/targets[i])
        else:
            qerror.append(targets[i]/(preds[i] + 0.1))
    return torch.mean(torch.cat(qerror) ** 2)

def print_loss(estimates, targets):
    esti = torch.exp(estimates)
#     print (torch.cat((estimates, esti, targets), 1))
    qerror = []
    for i in range(esti.shape[0]):
        if esti[i] > targets[i] + 0.1:
            qerror.append((esti[i] / (targets[i] + 0.1)).item())
        else:
            qerror.append(((targets[i] + 0.1) / esti[i]).item())
    
    return F.mse_loss(esti, targets), np.mean(qerror), np.max(qerror)

## Hyper-parameters Selection Methods

In [9]:
from random import sample

def construct_model():
    errors = []
    next_cnn_parameterss = []
    next_cnn_modelss = []
    next_output_models = []
    threshold_models = []
    for clus in range(100):
        print ('Begin Cluster: {}'.format(clus))
        idx = clus
        train = train_loaders[idx]
        test = test_loaders[idx]
        mini = min_cards[idx]
        maxi = max_cards[idx]
        prev_best_error = 100000.0
        cnn_parameters, cnn_models = [], []
        episode = 5
        queries_dimension = 200
        threshold_model = Threshold_Model()
        error, next_cnn_parameters, next_cnn_models,next_output_model = select_best_layer(prev_best_error, cnn_parameters, cnn_models, threshold_model, train, test, episode, queries_dimension)
        saved_error, saved_next_cnn_parameters, saved_next_cnn_models,saved_next_output_model = error, next_cnn_parameters, next_cnn_models,next_output_model
        while error is not None:
            saved_error, saved_next_cnn_parameters, saved_next_cnn_models,saved_next_output_model = error, next_cnn_parameters, next_cnn_models,next_output_model
            print ('Cluster: {}, Error: {}, CNN Layer Num: {}, Added CNN Layer: {}'.format(clus, error, len(next_cnn_parameters), next_cnn_parameters[-1]))
            error, next_cnn_parameters, next_cnn_models,next_output_model = select_best_layer(error, next_cnn_parameters, next_cnn_models, threshold_model, train, test, episode, queries_dimension)
        errors.append(saved_error)
        next_cnn_parameterss.append(saved_next_cnn_parameters)
        next_cnn_modelss.append(saved_next_cnn_models)
        next_output_models.append(saved_next_output_model)
        threshold_models.append(threshold_model)
    return errors, next_cnn_parameterss, next_cnn_modelss, next_output_models, threshold_models

class TunableParameters():
    
    def __init__(self, out_channel, kernel_size, stride, padding, pool_size, pool_type):
        self.out_channel = out_channel
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        self.pool_size = pool_size
        self.pool_type = pool_type
    
    def __repr__(self):
        return str(self.out_channel) +' '+ str(self.kernel_size) +' '+ str(self.stride) +' '+ str(self.padding) +' '+ str(self.pool_size) +' '+ str(self.pool_type)
 
    def __str__(self):
        return str(self.out_channel) +' '+ str(self.kernel_size) +' '+ str(self.stride) +' '+ str(self.padding) +' '+ str(self.pool_size) +' '+ str(self.pool_type)

def select_best_layer(prev_best_error, cnn_parameters, cnn_models, threshold_model, train, test, episode, queries_dimension):
    print ('Input Model Size: {}'.format(len(cnn_parameters)))
    if len(cnn_parameters) > 0:
        in_channel = cnn_parameters[-1].out_channel
    else:
        in_channel = 1
    in_size = queries_dimension
    for para in cnn_parameters:
        in_size = int((int((in_size - para.kernel_size + 2*(para.padding)) / para.stride) + 1) / para.pool_size)
        print(para.kernel_size, para.padding, para.stride, para.pool_size, in_size)
    
    if in_size < 10 or len(cnn_parameters) > 5:
        return None, None, None, None
    
    current_paras = []
    current_paras.append(TunableParameters(8,10,1,3,10,0))
    current_paras.append(TunableParameters(4,5,3,2,5,0))
#     current_paras.append(TunableParameters(4,3,1,0,3,0))
    current_paras.append(TunableParameters(2,2,1,0,2,0))
    
#     for out_channel in [2,4,8]:
#         for kernel_size in [2,4,8]:
#             for stride in range(1, min(4, kernel_size)):
#                 for padding in [0,2]:
#                     for pool_size in [kernel_size,]:
#                         for pool_type in [0,]:
#                             current_paras.append(TunableParameters(out_channel,kernel_size,stride,padding,pool_size,pool_type))
    print ('Group of parameters: {}'.format(len(current_paras)))
    next_cnn_models = []
    next_cnn_parameters = []
    next_output_model = None
#     current_paras = sample(current_paras, 2)
    for para in current_paras:
        print (para)
        in_size_local = int((int((in_size - para.kernel_size + 2*(para.padding)) / para.stride) + 1) / para.pool_size)
        if in_size_local < 10:
            continue
        print (in_size_local, para.out_channel)
        output_model = Output_Model(in_size_local * para.out_channel)
        added_cnn_layer = CNN_Model(in_channel, para.out_channel, para.kernel_size, para.stride, para.padding, para.pool_type, para.pool_size)
        paras = [{"params": model.parameters()} for model in cnn_models]
        paras.append({"params": added_cnn_layer.parameters()})
        paras.append({"params": threshold_model.parameters()})
        paras.append({"params": output_model.parameters()})
        opt = optim.Adam(paras, lr=0.001)
        new_cnn_models = []
        for model in cnn_models:
            new_cnn_models.append(model)
        new_cnn_models.append(added_cnn_layer)
        error = train_and_test(new_cnn_models, threshold_model, output_model, opt, train, test, episode)
        if error < prev_best_error - 0.1:
            prev_best_error = error
            new_cnn_parameters = []
            for para_old in cnn_parameters:
                new_cnn_parameters.append(para_old)
            next_output_model = output_model
            new_cnn_parameters.append(para)
            print ('Update layer: {}'.format(len(new_cnn_parameters)))
            next_cnn_parameters = new_cnn_parameters
            next_cnn_models = new_cnn_models
    if len(next_cnn_models) == 0:
        return None, None, None, None
    return prev_best_error, next_cnn_parameters, next_cnn_models, next_output_model
        

def only_test(cnn_models, threshold_model, output_model, test):
    for model in cnn_models:
        model.eval()
    threshold_model.eval()
    output_model.eval()
    q_errors = []
    for batch_idx, (queries, _, thresholds, targets) in enumerate(test):
        queries = Variable(queries)
        thresholds = Variable(thresholds)
        targets = Variable(targets)

        queries = queries.unsqueeze(2).permute(0,2,1)
        for model in cnn_models:
            queries = model(queries)
        threshold = threshold_model(thresholds)
        queries = queries.view(queries.shape[0], -1)
        estimates = output_model(queries, threshold)

        loss = l1_loss(estimates, targets)
        
        esti = torch.exp(estimates)
        for i in range(esti.shape[0]):
            if esti[i] > targets[i] + 0.1:
                q_errors.append((esti[i] / (targets[i] + 0.1)).item())
            else:
                q_errors.append(((targets[i] + 0.1) / esti[i]).item())
    mean = np.mean(q_errors)
    percent90 = np.percentile(q_errors, 90)
    percent95 = np.percentile(q_errors, 95)
    percent99 = np.percentile(q_errors, 99)
    median = np.median(q_errors)
    maxi = np.max(q_errors)
    print ('Testing: Mean Error {}, Median Error {}, 90 Percent {}, 95 Percent {}, 99 Percent {}, Max Percent {}'
           .format(mean, median, percent90, percent95, percent99, maxi))
    
    
def train_and_test(cnn_models, threshold_model, output_model, opt, train, test, episode):
    print ('size: {}'.format(len(train)))
    test_errors = []
    for e in range(episode):
        for model in cnn_models:
            model.train()
        threshold_model.train()
        output_model.train()
        for batch_idx, (queries, _, thresholds, targets) in enumerate(train):
    #         print (torch.cat((queries, thresholds), 1)[0])
            queries = Variable(queries)
            thresholds = Variable(thresholds)
            targets = Variable(targets)
    #         print (targets)
            opt.zero_grad()
            queries = queries.unsqueeze(2).permute(0,2,1)
            for model in cnn_models:
                queries = model(queries)
            threshold = threshold_model(thresholds)
            queries = queries.view(queries.shape[0], -1)
            estimates = output_model(queries, threshold)
            
            loss = l1_loss(estimates, targets)
            loss.backward()
            opt.step()
            for p in model.parameters():
                p.data.clamp_(-10, 10)
            next(threshold_model.fc1.parameters()).data.clamp_(0)
            next(threshold_model.fc2.parameters()).data.clamp_(0)
            next(output_model.fc2.parameters()).data.clamp_(0)
#             if batch_idx % 100 == 0:
#                 print('Training: Iteration {0}, Batch {1}, Loss {2}'.format(e, batch_idx, loss.item()))
#                 print(cnn_models[0].layer[0].weight.grad)
        for model in cnn_models:
            model.eval()
        threshold_model.eval()
        output_model.eval()
        test_loss = 0.0
        mse_error = 0.0
        q_mean = 0.0
        q_max = 0.0
        for batch_idx, (queries, _, thresholds, targets) in enumerate(test):
            queries = Variable(queries)
            thresholds = Variable(thresholds)
            targets = Variable(targets)
            
            queries = queries.unsqueeze(2).permute(0,2,1)
            for model in cnn_models:
                queries = model(queries)
            threshold = threshold_model(thresholds)
            queries = queries.view(queries.shape[0], -1)
            estimates = output_model(queries, threshold)
            
            loss = l1_loss(estimates, targets)
            mse, qer_mean, qer_max = print_loss(estimates, targets)
            test_loss += loss.item()
            mse_error += mse.item()
            q_mean += qer_mean
            if qer_max > q_max:
                q_max = qer_max
        test_loss /= len(test)
        mse_error /= len(test)
        q_mean /= len(test)
        test_errors.append(q_mean)
        print ('Testing: Iteration {0}, Loss {1}, MSE_error {2}, Q_error_mean {3}, Q_error_max {4}'.format(e, test_loss, mse_error, q_mean, q_max))
    return np.mean(test_errors[-3:])

## Select Model Hyper-parameters

In [10]:
errors, next_cnn_parameterss, next_cnn_modelss, next_output_models, threshold_models = construct_model()

Begin Cluster: 0
Input Model Size: 0
Group of parameters: 3
8 10 1 3 10 0
19 8
size: 1017
Testing: Iteration 0, Loss 18.25792927087522, MSE_error 56338708.15686274, Q_error_mean 18.257929357747937, Q_error_max 522.2666015625
Testing: Iteration 1, Loss 11.40807912676942, MSE_error 50657012.17254902, Q_error_mean 11.40807909487842, Q_error_max 395.922607421875
Testing: Iteration 2, Loss 3.541655423594456, MSE_error 15240197.619607843, Q_error_mean 3.5416553597471196, Q_error_max 279.26727294921875
Testing: Iteration 3, Loss 2.6108544237473432, MSE_error 92948083.21960784, Q_error_mean 2.6108544049001337, Q_error_max 174.2854461669922
Testing: Iteration 4, Loss 2.5073244052774766, MSE_error 237909529.6470588, Q_error_mean 2.5073243962775327, Q_error_max 128.38766479492188
Update layer: 1
4 5 3 2 5 0
13 4
size: 1017
Testing: Iteration 0, Loss 4.511531195921056, MSE_error 25138849.149019606, Q_error_mean 4.511531252887157, Q_error_max 309.031494140625
Testing: Iteration 1, Loss 2.7875090627

Testing: Iteration 3, Loss 2.779797821533023, MSE_error 253240175.78346458, Q_error_mean 2.779797860600905, Q_error_max 180.92367553710938
Testing: Iteration 4, Loss 2.6978376029044626, MSE_error 452847923.9055118, Q_error_mean 2.697837555979528, Q_error_max 206.40489196777344
Update layer: 1
Cluster: 2, Error: 2.7412460724532295, CNN Layer Num: 1, Added CNN Layer: 2 2 1 0 2 0
Input Model Size: 1
2 0 1 2 99
Group of parameters: 3
8 10 1 3 10 0
4 5 3 2 5 0
2 2 1 0 2 0
49 2
size: 1014
Testing: Iteration 0, Loss 3.6043058652577438, MSE_error 36583908.37401575, Q_error_mean 3.604305845012216, Q_error_max 446.0040283203125
Testing: Iteration 1, Loss 3.1654903231643314, MSE_error 267139069.53543308, Q_error_mean 3.165490332556128, Q_error_max 448.4348449707031
Testing: Iteration 2, Loss 3.0801864393114107, MSE_error 314525910.6535433, Q_error_mean 3.0801864445845397, Q_error_max 410.294921875
Testing: Iteration 3, Loss 3.023546682568047, MSE_error 361996240.3031496, Q_error_mean 3.0235467107

Testing: Iteration 0, Loss 3.522507491590675, MSE_error 606258.0896561135, Q_error_mean 3.522507543719524, Q_error_max 215.57925415039062
Testing: Iteration 1, Loss 3.019639092241312, MSE_error 2055278.9209879912, Q_error_mean 3.019639099879221, Q_error_max 180.94125366210938
Testing: Iteration 2, Loss 2.9571433868991237, MSE_error 3925066.34566048, Q_error_mean 2.957143337873452, Q_error_max 171.9108123779297
Testing: Iteration 3, Loss 2.8725589165000414, MSE_error 2472799.7414028384, Q_error_mean 2.8725588885437214, Q_error_max 141.80526733398438
Testing: Iteration 4, Loss 2.8898056834025154, MSE_error 2117866.7677401747, Q_error_mean 2.8898056734417126, Q_error_max 125.80035400390625
2 2 1 0 2 0
99 2
size: 912
Testing: Iteration 0, Loss 2.7880401111586126, MSE_error 747394.4744814411, Q_error_mean 2.788040090587427, Q_error_max 146.79931640625
Testing: Iteration 1, Loss 2.364847595514689, MSE_error 12140773.233078603, Q_error_mean 2.3648476039564694, Q_error_max 105.70518493652344
T

Testing: Iteration 3, Loss 2.745077407531191, MSE_error 54458931.24386962, Q_error_mean 2.745077373197107, Q_error_max 159.010498046875
Testing: Iteration 4, Loss 2.683530143573524, MSE_error 92607391.246561, Q_error_mean 2.683530103574779, Q_error_max 137.67466735839844
Begin Cluster: 8
Input Model Size: 0
Group of parameters: 3
8 10 1 3 10 0
19 8
size: 946
Testing: Iteration 0, Loss 16.100284628727266, MSE_error 6374312.124472573, Q_error_mean 16.100284433855073, Q_error_max 210.96298217773438
Testing: Iteration 1, Loss 6.375695200409064, MSE_error 4439394.285864979, Q_error_mean 6.37569511196057, Q_error_max 223.2471466064453
Testing: Iteration 2, Loss 3.2936530133339925, MSE_error 30840365.536392406, Q_error_mean 3.2936530021641945, Q_error_max 191.68714904785156
Testing: Iteration 3, Loss 3.251480895255688, MSE_error 70622596.02531646, Q_error_mean 3.251480899151739, Q_error_max 207.74331665039062
Testing: Iteration 4, Loss 3.1587399510894647, MSE_error 70059475.0464135, Q_error_m

Testing: Iteration 2, Loss 2.9795079297753446, MSE_error 217693379.0119522, Q_error_mean 2.9795079183426685, Q_error_max 109.11905670166016
Testing: Iteration 3, Loss 2.899460001295781, MSE_error 111369132.03187251, Q_error_mean 2.899459976525611, Q_error_max 92.4363021850586
Testing: Iteration 4, Loss 2.821825667681447, MSE_error 175135730.0, Q_error_mean 2.821825708861572, Q_error_max 102.76632690429688
Update layer: 1
Cluster: 10, Error: 2.900264534576617, CNN Layer Num: 1, Added CNN Layer: 2 2 1 0 2 0
Input Model Size: 1
2 0 1 2 99
Group of parameters: 3
8 10 1 3 10 0
4 5 3 2 5 0
2 2 1 0 2 0
49 2
size: 996
Testing: Iteration 0, Loss 3.7566851638702756, MSE_error 23490971.191235058, Q_error_mean 3.7566851201077305, Q_error_max 299.45660400390625
Testing: Iteration 1, Loss 3.2714031312570153, MSE_error 115418338.61752988, Q_error_mean 3.2714031955344214, Q_error_max 214.35711669921875
Testing: Iteration 2, Loss 3.2026816535280993, MSE_error 133824344.73306774, Q_error_mean 3.20268162

...

Testing: Iteration 1, Loss 2.9655833374201555, MSE_error 30732284.747081712, Q_error_mean 2.965583321521455, Q_error_max 539.3544311523438
Testing: Iteration 2, Loss 2.7638874916714915, MSE_error 44943448.019455254, Q_error_mean 2.7638874988648467, Q_error_max 393.1244812011719
Testing: Iteration 3, Loss 2.7044395398537007, MSE_error 124634991.74319066, Q_error_mean 2.7044395672816286, Q_error_max 379.2299499511719
Testing: Iteration 4, Loss 2.6138732275610304, MSE_error 48976634.55642024, Q_error_mean 2.6138732300637084, Q_error_max 183.0997314453125
2 2 1 0 2 0
99 2
size: 1014
Testing: Iteration 0, Loss 4.1531351289860465, MSE_error 13670509.091439689, Q_error_mean 4.1531351046268385, Q_error_max 265.0863952636719
Testing: Iteration 1, Loss 2.7655314629179957, MSE_error 11937587.998054475, Q_error_mean 2.7655314858951434, Q_error_max 256.0559387207031
Testing: Iteration 2, Loss 2.3715225908079036, MSE_error 65581886.64396887, Q_error_mean 2.3715225972575844, Q_error_max 204.783889770

Testing: Iteration 3, Loss 2.4301363906217, MSE_error 107643413.82936507, Q_error_mean 2.430136433893993, Q_error_max 74.09859466552734
Testing: Iteration 4, Loss 2.446259511841668, MSE_error 164469301.4642857, Q_error_mean 2.4462595065137225, Q_error_max 89.21764373779297
Begin Cluster: 90
Input Model Size: 0
Group of parameters: 3
8 10 1 3 10 0
19 8
size: 1018
Testing: Iteration 0, Loss 13.678199246699707, MSE_error 94052016.6848249, Q_error_mean 13.67819927237812, Q_error_max 720.0165405273438
Testing: Iteration 1, Loss 3.860684766843625, MSE_error 34686815.03112841, Q_error_mean 3.860684767106878, Q_error_max 583.8177490234375
Testing: Iteration 2, Loss 3.4303934063892885, MSE_error 161485193.54085603, Q_error_mean 3.4303933401104962, Q_error_max 348.68963623046875
Testing: Iteration 3, Loss 3.1094246309554996, MSE_error 405064549.6498054, Q_error_mean 3.1094246694634142, Q_error_max 373.5863342285156
Testing: Iteration 4, Loss 2.9979346420050597, MSE_error 421110253.8988327, Q_err

Testing: Iteration 0, Loss 5.402074549879346, MSE_error 27859300.5, Q_error_mean 5.402074575433609, Q_error_max 750.3577880859375
Testing: Iteration 1, Loss 4.079005596183595, MSE_error 246488358.85714287, Q_error_mean 4.079005572831869, Q_error_max 565.2315063476562
Testing: Iteration 2, Loss 3.840271330068982, MSE_error 1202962037.2063491, Q_error_mean 3.8402713476924917, Q_error_max 284.8792419433594
Testing: Iteration 3, Loss 3.6282239263019864, MSE_error 837392660.4126984, Q_error_mean 3.6282239971219106, Q_error_max 238.8544464111328
Testing: Iteration 4, Loss 3.6441336718816606, MSE_error 810695188.4444444, Q_error_mean 3.6441336886899713, Q_error_max 181.900390625
Begin Cluster: 93
Input Model Size: 0
Group of parameters: 3
8 10 1 3 10 0
19 8
size: 1017
Testing: Iteration 0, Loss 17.560289720850665, MSE_error 119722298.11023623, Q_error_mean 17.56028987432561, Q_error_max 711.2115478515625
Testing: Iteration 1, Loss 5.429989213079918, MSE_error 53135523.30708662, Q_error_mean 5

Testing: Iteration 4, Loss 3.4111320266723633, MSE_error 27990311.17, Q_error_mean 3.4111320831341048, Q_error_max 360.606689453125
2 2 1 0 2 0
99 2
size: 1008
Testing: Iteration 0, Loss 4.898732782363892, MSE_error 11945904.7125, Q_error_mean 4.898732789407174, Q_error_max 245.90089416503906
Testing: Iteration 1, Loss 2.8193424701690675, MSE_error 22493111.5320625, Q_error_mean 2.8193424906258784, Q_error_max 178.0536651611328
Testing: Iteration 2, Loss 2.5577203235626222, MSE_error 87723657.60635157, Q_error_mean 2.5577203133205573, Q_error_max 116.21702575683594
Testing: Iteration 3, Loss 2.529076340675354, MSE_error 93217351.38963996, Q_error_mean 2.529076354421675, Q_error_max 97.6091079711914
Testing: Iteration 4, Loss 2.5188337683677675, MSE_error 180317310.688, Q_error_mean 2.518833762212346, Q_error_max 80.7171630859375
Update layer: 1
Cluster: 95, Error: 2.5352101433181926, CNN Layer Num: 1, Added CNN Layer: 2 2 1 0 2 0
Input Model Size: 1
2 0 1 2 99
Group of parameters: 3
8 

Testing: Iteration 1, Loss 8.79504170115032, MSE_error 83825345.53968254, Q_error_mean 8.795041619249897, Q_error_max 443.5504150390625
Testing: Iteration 2, Loss 3.790143506867545, MSE_error 169465545.2142857, Q_error_mean 3.7901435368103047, Q_error_max 343.4941101074219
Testing: Iteration 3, Loss 3.435952507314228, MSE_error 501916254.04761904, Q_error_mean 3.4359524929674254, Q_error_max 242.17193603515625
Testing: Iteration 4, Loss 3.6080472393641396, MSE_error 229648964.03174603, Q_error_mean 3.608047234569201, Q_error_max 173.64871215820312
Update layer: 1
4 5 3 2 5 0
13 4
size: 1020
Testing: Iteration 0, Loss 8.073731043982127, MSE_error 68081189.3968254, Q_error_mean 8.073731109799285, Q_error_max 1030.207763671875
Testing: Iteration 1, Loss 5.4620036416583595, MSE_error 30446246.523809522, Q_error_mean 5.462003578514794, Q_error_max 1190.192138671875
Testing: Iteration 2, Loss 4.157817874635969, MSE_error 151830239.34920636, Q_error_mean 4.157817894915465, Q_error_max 595.803

## Final Training

In [12]:
for idx in range(100):
    print (idx)
    test = test_loaders[idx]
    train = train_loaders[idx]
    paras = [{"params": model.parameters()} for model in next_cnn_modelss[idx]]
    paras.append({"params": threshold_models[idx].parameters()})
    paras.append({"params": next_output_models[idx].parameters()})
    opt = optim.Adam(paras, lr=0.001)
    episode = 3
    train_and_test(next_cnn_modelss[idx], threshold_models[idx], next_output_models[idx], opt, train, test, episode)
    only_test(next_cnn_modelss[idx], threshold_models[idx], next_output_models[idx], test)

0
size: 1017
Testing: Iteration 0, Loss 2.347348792412702, MSE_error 362330775.7176471, Q_error_mean 2.3473488030249094, Q_error_max 89.67131042480469
Testing: Iteration 1, Loss 2.3014326460221235, MSE_error 158471470.76862746, Q_error_mean 2.301432655868577, Q_error_max 60.97597122192383
Testing: Iteration 2, Loss 2.28870692393359, MSE_error 213861622.62745097, Q_error_mean 2.2887069143391416, Q_error_max 67.36402893066406
Testing: Mean Error 2.2857665124045874, Median Error 1.867579698562622, 90 Percent 3.554525899887085, 95 Percent 4.558394360542297, 99 Percent 9.262087574005122, Max Percent 67.36402893066406
1
size: 970
Testing: Iteration 0, Loss 2.1374128875418457, MSE_error 32441397.557613168, Q_error_mean 2.1374128812865316, Q_error_max 35.43651580810547
Testing: Iteration 1, Loss 2.0297334272675065, MSE_error 16279927.74691358, Q_error_mean 2.029733442788833, Q_error_max 27.261947631835938
Testing: Iteration 2, Loss 2.035033245145539, MSE_error 13243731.001543209, Q_error_mean 

Testing: Iteration 1, Loss 2.258133311884119, MSE_error 25171666.653526377, Q_error_mean 2.2581333361453493, Q_error_max 74.26776885986328
Testing: Iteration 2, Loss 2.2496266578315596, MSE_error 17515556.75365539, Q_error_mean 2.249626697674343, Q_error_max 61.761260986328125
Testing: Mean Error 2.2496286390587694, Median Error 1.6340752840042114, 90 Percent 3.9589386701583873, 95 Percent 5.545750236511228, 99 Percent 10.376703624725357, Max Percent 61.761260986328125
14
size: 1006
Testing: Iteration 0, Loss 2.9233955577075244, MSE_error 413747188.5179283, Q_error_mean 2.923395555356297, Q_error_max 180.9786376953125
Testing: Iteration 1, Loss 2.879334873412235, MSE_error 341086859.55378485, Q_error_mean 2.879334868933843, Q_error_max 187.3547821044922
Testing: Iteration 2, Loss 2.865045138089305, MSE_error 325869698.4063745, Q_error_mean 2.8650450859373957, Q_error_max 186.781494140625
Testing: Mean Error 2.8657754750671467, Median Error 2.1166199445724487, 90 Percent 4.8265021800994

Testing: Iteration 2, Loss 2.243657992068065, MSE_error 87255418.22891566, Q_error_mean 2.243657995992674, Q_error_max 121.46820831298828
Testing: Mean Error 2.243207412040589, Median Error 1.659825086593628, 90 Percent 3.690701818466187, 95 Percent 5.335345458984371, 99 Percent 10.613204689025874, Max Percent 121.46820831298828
27
size: 663
Testing: Iteration 0, Loss 2.6253201861581403, MSE_error 4994794.378192833, Q_error_mean 2.625320159103552, Q_error_max 198.5837860107422
Testing: Iteration 1, Loss 2.707670077592313, MSE_error 4762651.481310816, Q_error_mean 2.707670007001588, Q_error_max 165.5523223876953
Testing: Iteration 2, Loss 2.6908838135039734, MSE_error 3865610.3875315776, Q_error_mean 2.690883772643972, Q_error_max 160.18994140625
Testing: Mean Error 2.6922596764467035, Median Error 1.8633933067321777, 90 Percent 4.908078956604005, 95 Percent 6.821052169799803, 99 Percent 13.733142681121837, Max Percent 160.18994140625
28
size: 981
Testing: Iteration 0, Loss 2.4256254420

Testing: Mean Error 2.139015842564967, Median Error 1.5107399225234985, 90 Percent 3.5663065195083608, 95 Percent 5.107502388954156, 99 Percent 10.499786787033067, Max Percent 78.04794311523438
40
size: 979
Testing: Iteration 0, Loss 1.8274521415795737, MSE_error 3932810.4273373983, Q_error_mean 1.827452118035651, Q_error_max 30.51961326599121
Testing: Iteration 1, Loss 1.8189046639737074, MSE_error 5109274.778963415, Q_error_mean 1.8189046833982436, Q_error_max 27.686548233032227
Testing: Iteration 2, Loss 1.8395728013379786, MSE_error 6095345.959857724, Q_error_mean 1.8395728234860165, Q_error_max 28.697837829589844
Testing: Mean Error 1.8396012112488818, Median Error 1.4200488328933716, 90 Percent 2.8390645503997805, 95 Percent 4.035575819015501, 99 Percent 8.441703681945791, Max Percent 28.697837829589844
41
size: 1001
Testing: Iteration 0, Loss 2.697390303668748, MSE_error 341836265.3466135, Q_error_mean 2.6973902976850312, Q_error_max 66.91948699951172
Testing: Iteration 1, Loss 

Testing: Iteration 0, Loss 2.4819524476650674, MSE_error 218884266.04743084, Q_error_mean 2.4819524552925585, Q_error_max 58.21034240722656
Testing: Iteration 1, Loss 2.4237492918497017, MSE_error 336047831.4189723, Q_error_mean 2.4237492549101267, Q_error_max 69.68434143066406
Testing: Iteration 2, Loss 2.4725140912730703, MSE_error 722714901.2806324, Q_error_mean 2.4725140734313467, Q_error_max 82.72364044189453
Testing: Mean Error 2.472438295481576, Median Error 1.7789218425750732, 90 Percent 4.194965934753418, 95 Percent 5.965913963317869, 99 Percent 13.103820724487303, Max Percent 82.72364044189453
54
size: 969
Testing: Iteration 0, Loss 4.867970884338884, MSE_error 393578601.11570245, Q_error_mean 4.867970890496463, Q_error_max 1322.1463623046875
Testing: Iteration 1, Loss 4.7685937349461325, MSE_error 455968149.1404959, Q_error_mean 4.768593632835576, Q_error_max 1191.4140625
Testing: Iteration 2, Loss 4.614715437258571, MSE_error 223195545.79752067, Q_error_mean 4.6147154319767

Testing: Iteration 1, Loss 2.5798682910037707, MSE_error 264222656.91633466, Q_error_mean 2.5798682956973056, Q_error_max 76.40616607666016
Testing: Iteration 2, Loss 2.620623242807578, MSE_error 574435847.0278884, Q_error_mean 2.6206232259109865, Q_error_max 92.75137329101562
Testing: Mean Error 2.6205491740612588, Median Error 1.7872676253318787, 90 Percent 4.459729099273685, 95 Percent 6.843051362037653, 99 Percent 15.35264882087709, Max Percent 92.75137329101562
67
size: 974
Testing: Iteration 0, Loss 2.575493870448674, MSE_error 22287124.91718107, Q_error_mean 2.5754938788533632, Q_error_max 95.93360900878906
Testing: Iteration 1, Loss 2.475372915895878, MSE_error 50342565.83744856, Q_error_mean 2.4753729138144958, Q_error_max 121.63003540039062
Testing: Iteration 2, Loss 2.519922175034574, MSE_error 25432852.997942388, Q_error_mean 2.5199221684264694, Q_error_max 105.88711547851562
Testing: Mean Error 2.5199107744438662, Median Error 2.0065791606903076, 90 Percent 4.3327686786651

Testing: Iteration 2, Loss 2.355327551790293, MSE_error 56783939.05497925, Q_error_mean 2.35532756741493, Q_error_max 44.882381439208984
Testing: Mean Error 2.3547994032080766, Median Error 1.7967966794967651, 90 Percent 3.9698292255401615, 95 Percent 5.337073707580558, 99 Percent 10.416359729766851, Max Percent 44.882381439208984
80
size: 994
Testing: Iteration 0, Loss 3.3118702047239474, MSE_error 65458243.54471545, Q_error_mean 3.3118701781343094, Q_error_max 154.8776397705078
Testing: Iteration 1, Loss 3.2677547447080535, MSE_error 95269731.78252032, Q_error_mean 3.2677547517588468, Q_error_max 193.4156494140625
Testing: Iteration 2, Loss 3.2547838309916055, MSE_error 76704701.09552845, Q_error_mean 3.2547838004169667, Q_error_max 191.8903350830078
Testing: Mean Error 3.254810790229738, Median Error 2.204453945159912, 90 Percent 5.636144399642945, 95 Percent 8.12903628349304, 99 Percent 20.148892612457274, Max Percent 191.8903350830078
81
size: 1014
Testing: Iteration 0, Loss 3.104

Testing: Mean Error 3.1309674551628808, Median Error 2.349545955657959, 90 Percent 5.5947964668273915, 95 Percent 7.27034783363341, 99 Percent 14.29929372787474, Max Percent 72.91023254394531
93
size: 1017
Testing: Iteration 0, Loss 2.8737653719158622, MSE_error 498158558.6929134, Q_error_mean 2.87376534632492, Q_error_max 166.58982849121094
Testing: Iteration 1, Loss 2.9108336281588696, MSE_error 471853162.2992126, Q_error_mean 2.910833600413, Q_error_max 147.06358337402344
Testing: Iteration 2, Loss 2.8280999763744084, MSE_error 639460642.8031496, Q_error_mean 2.828099929963326, Q_error_max 141.6313934326172
Testing: Mean Error 2.8308175721869984, Median Error 2.139757752418518, 90 Percent 4.671738576889037, 95 Percent 6.1784117937087855, 99 Percent 14.162508268356243, Max Percent 141.6313934326172
94
size: 937
Testing: Iteration 0, Loss 1.9879641123747422, MSE_error 7354240.424258474, Q_error_mean 1.9879641174457146, Q_error_max 65.09520721435547
Testing: Iteration 1, Loss 1.9348636

## Save The Model

In [13]:
for idx in range(100):
    states = {}
    for idd, cnn_model in enumerate(next_cnn_modelss[idx]):
        states['cnn_model_state_dict_' + str(idd)] = cnn_model.state_dict()
    states['threshold_model_state_dict'] = threshold_models[idx].state_dict()
    states['output_model_state_dict'] = next_output_models[idx].state_dict()
    torch.save(states, '/home/sunji/ANN/glove_200_angular/saved_models/local_glove_200_angular_cluster_' + str(idx) + '.model')
    

In [14]:
import pickle
with open('/home/sunji/ANN/glove_200_angular/saved_models/cnn_hyper_parameters.hyperpara', 'w') as handle:
    for idx in range(100):
        handle.write(';'.join(str(x) for x in next_cnn_parameterss[idx]))
        handle.write('\n')

## Model Test

In [ ]:
model = Model()
model.load_state_dict(torch.load('MDN_model'))
model.eval()